# CS6140 Assignments

**Instructions**
1. In each assignment cell, look for the block:
 ```
  #BEGIN YOUR CODE
  raise NotImplementedError.new()
  #END YOUR CODE
 ```
1. Replace this block with your solution.
1. Test your solution by running the cells following your block (indicated by ##TEST##)
1. Click the "Validate" button above to validate the work.

**Notes**
* You may add other cells and functions as needed
* Keep all code in the same notebook
* In order to receive credit, code must "Validate" on the JupyterHub server

---

# Assignment 3: Gradient Descent and Maximum Likelihood

In [1]:
require './assignment_lib'

"if(window['d3'] === undefined ||\n   window['Nyaplot'] === undefined){\n    var path = {\"d3\":\"https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.5/d3.min\",\"downloadable\":\"http://cdn.rawgit.com/domitry/d3-downloadable/master/d3-downloadable\"};\n\n\n\n    var shim = {\"d3\":{\"exports\":\"d3\"},\"downloadable\":{\"exports\":\"downloadable\"}};\n\n    require.config({paths: path, shim:shim});\n\n\nrequire(['d3'], function(d3){window['d3']=d3;console.log('finished loading d3');require(['downloadable'], function(downloadable){window['downloadable']=downloadable;console.log('finished loading downloadable');\n\n\tvar script = d3.select(\"head\")\n\t    .append(\"script\")\n\t    .attr(\"src\", \"http://cdn.rawgit.com/domitry/Nyaplotjs/master/release/nyaplot.js\")\n\t    .attr(\"async\", true);\n\n\tscript[0][0].onload = script[0][0].onreadystatechange = function(){\n\n\n\t    var event = document.createEvent(\"HTMLEvents\");\n\t    event.initEvent(\"load_nyaplot\",false,false);\n\t    win

true

## Question 1.1 (5 points)

Let's implement a test function for the gradient descent optimizer, a 3D simple parabola. All gradient-optimized trainers are implemented as a objective function. The follow the same basic pattern:

```ruby
class MyGradientLearnableModel
  def func x, w
    #Returns the value of the objective function, 
    #  summing across all examples in x
  end
  def grad x, w
    #Returns a hash of derivative values for each variable in w,
    # gradient is summed across all examples in x
    dw = {"0" => w["0"], "1" => w["1"]}
  end
  def adjust w
    # Applies any problem-specific alterations to w
  end
end
```

Now, let's implement a Parabola objective function which does not depend on the data at all. It is defined as follows:

### $L(w) = \frac{1}{2}\left( \left(w_{0} - 1\right)^2 + \left(w_{1} - 2\right)^2 \right)$

Implement the ```func``` method for the loss function, $L(w)$. 

**Note**: The data, i.e., ```x``` is not actually used in this objective function. Other objective functions may use ```x```. 

In [2]:
class ParabolaObjective
  def func x, w
    # BEGIN YOUR CODE
    if w.length == 2
          return 0.5*((w[w.keys[0]]-1.0)**2+(w[w.keys[1]]-2.0)**2)
    elsif w.length == 1
          return 0.5*((w[w.keys[0]]-1.0)**2)
    end
    #END YOUR CODE
  end 
  def adjust w
  end
end

:adjust

In [3]:
### TESTS ###
t1 = ParabolaObjective.new
assert_in_delta(0.0, t1.func([], {"0" => 1.0, "1" => 2.0}), 1e-3)
assert_in_delta(0.5, t1.func([], {"0" => 1.0, "1" => 1.0}), 1e-3)
assert_in_delta(0.5, t1.func([], {"0" => 1.0, "1" => 3.0}), 1e-3)
assert_in_delta(2.5, t1.func([], {"0" => 3.0, "1" => 1.0}), 1e-3)

## Question 1.2 (5 Points)

Implement the gradient function for $L(w)$. It evaluates the gradient for the value of $x$ for each dimension of $w$. In this simple case, $L(w)$ does not depend on $x$.

In [4]:
class ParabolaObjective
  def grad x, w
    # BEGIN YOUR CODE
        
    dw = Hash.new()
    name = w.keys
    if w.length == 2
        dw = {name[0] => w[name[0]]-1.0, name[1] => w[name[1]]-2.0}
    elsif w.length == 1
        dw = {name[0] => w[name[0]]-1.0}
    end
    return dw
    #END YOUR CODE
  end
end

:grad

In [5]:
### TESTS ###
t2 = ParabolaObjective.new
w2_1 = t2.grad([], {"0" => 3.0, "1" => 7.0})
assert_in_delta(2.0, w2_1["0"], 1e-3)
assert_in_delta(5.0, w2_1["1"], 1e-3)

w2_2 = t2.grad([], {"0" => -3.0, "1" => -7.0})
assert_in_delta(-4.0, w2_2["0"], 1e-3)
assert_in_delta(-9.0, w2_2["1"], 1e-3)


## Question 2.1 (1 Point)


Implement gradient descent for any objective function class. Your function must provide a callback which we will use to monitor its performance and possibly to halt execution. A simple example illustrating the callback is as follows:

```ruby
def gradient_descent_example dataset, w, obj, learning_rate, tol, max_iter, &block
    iter = 1
    until converged(last_loss, current_loss) do
        w_last = w
        loss = calc_loss(w)
        w = update(w)
        w = adjust(w)
        iter += 1
        break unless yield w, iter, loss
    end
    
    return w
end

```

There are three main parts to the algorithm above:
1. Detecting the convergence
1. The norm can be calculated as the dot product of two vectors: $||w|| = w \cdot w$
1. Once the loss and gradient functions are calculated, we will update the values of each weight

### Implement dot product
In this first part, implement the dot product. The dot product below should be for sparse vectors. Use ```has_key?``` to skip entries in vector ```w``` not present in ```x```.

In [6]:
#Implement the error function given a weight vector, w
def dot x, w
  # BEGIN YOUR CODE
   x_name = x.keys
    w_name = w.keys

    i = 0
    dot = 0.0
    if x_name.length == 0 
      dot =  0.0
    else 
        while i<x_name.length do

          if w_name.include?x_name[i]
            index = w_name.index { |x| [x_name[i]].include?(x) }
            if x[x_name[i]].is_a? Numeric  and w[w_name[index]].is_a? Numeric
              dot += x[x_name[i]]*w[w_name[index]]
            else 
              dot +=0.0
            end
          else 
            dot += 0.0
          end
          i+=1
        end  
    end
  return dot
  
  
  #END YOUR CODE
end


:dot

In [7]:
### TEST ###
assert_in_delta 6.0, dot({"a" => 2.0}, {"a" => 3.0}), 1e-6
assert_in_delta 6.0, dot({"a" => 2.0}, {"a" => 3.0, "b" => 4.0}), 1e-6
assert_equal 0.0, dot({}, {})
assert_equal 0.0, dot({"a" => 1.0}, {"b" => 1.0})

## Question 2.2 (1 Point)
Implement the L2 norm for a vector, i.e., $\left \lVert x \right \rVert$, when represented by a hash. Hint: use ```dot``` and don't forget the square root.

In [8]:
def norm w
  # BEGIN YOUR CODE
  return Math.sqrt(dot w, w)
  #END YOUR CODE
end

:norm

In [9]:
### TEST ###
assert_in_delta 2.0, norm({"a" => 1.41421, "b" => 1.41421}), 1e-2
assert_in_delta 2.0, norm({"a" => -1.41421, "b" => 1.41421}), 1e-2
assert_in_delta 0.0, norm({}), 1e-2

## Question 2.3 (3 points)
Implement a function that updates a weight vector, ```w```, given a gradient vector, ```dw```, and learning rate, ```lr```.  Do not change the original weight vector. Hint: use ```clone```.

In [10]:
def update_weights(w, dw, lr)
  # BEGIN YOUR CODE
  i = 0
  name = w.keys
  new_weights = Hash.new
  while i< w.length do
    new_weights.store(name[i],w[name[i]] - lr * dw[name[i]])
    i+=1
  end
  return new_weights
  #END YOUR CODE
end

:update_weights

In [11]:
### TEST ###
assert_in_delta 1.5, update_weights({"a" => 1.0}, {"a" => -0.25}, 2.0)["a"], 1e-2
assert_in_delta 2.5, update_weights({"a" => 1.0, "b" => 3.0}, {"a" => -0.25, "b" => 0.25}, 2.0)["b"], 1e-2

## Question 2.4 (15 Points)

Now, put all these functions together to implement gradient descent. This function takes the ```dataset``` and calls the function and gradient on all the examples. Hint: Increment ```iter``` before calling ```yield```.

To detect the convergence, we will continue iterating until one of these two conditions occurs:

1. Absolute difference between the objective function before and after parameter update is less than ```tol```.
1. We have run more than ```max_iter``` iterations. 

In [12]:
def gradient_descent dataset, w, obj, learning_rate, tol, max_iter, &block
  iter = 0
  examples = dataset["data"]
  # BEGIN YOUR CODE
    last_loss = 0.0
    loss = 100*tol
  
    until ((last_loss - loss).abs < tol) do 

        last_loss = obj.func(examples, w)
        dw = obj.grad(examples, w)
        w = update_weights(w, dw, learning_rate)
        loss = obj.func(examples, w)
      
        iter +=1
        break unless yield w, iter, loss
        break if iter > max_iter
    end

  
  #END YOUR CODE
  return w
end

:gradient_descent

In [13]:
### TESTS ###

t4 = ParabolaObjective.new
t4_w_init = {"0" => 3.0, "1" => 7.0}
t4_w_goal = {"0" => 1.0, "1" => 2.0}
t4_dataset = {"data" => []}

t4_loss = 1.0
t4_w = nil
gradient_descent t4_dataset, t4_w_init, t4, 0.1, 0.001, 100 do |w, iter, loss|
  t4_loss = loss
  t4_w = w
end

assert_in_delta 0.01, t4_loss, 1e-2
assert_in_delta 1.0, t4_w["0"], 1e-1
assert_in_delta 2.0, t4_w["1"], 1e-1

In [14]:
### TEST ###
t4 = ParabolaObjective.new
t4_w_init = {"0" => 3.0, "1" => 7.0}
t4_data = {"data" => []}

t4_total_loss = 0.0
t4_iterations = []
t4_losses = []
gradient_descent t4_data, t4_w_init, t4, 0.1, 0.001, 100 do |w, iter, loss|
  t4_total_loss += loss
  t4_iterations << iter
  assert_true(iter > 0, "Make sure to increment 'iter' before calling 'yield'")
  t4_losses << t4_total_loss / iter
end

assert_true(t4_iterations.size > 30)
assert_true(t4_losses[-1] < 3)
assert_true(t4_losses[-1] > 0)
assert_true(t4_iterations[-1] > 30)

In [15]:
### Plot the cumulative loss per iteration
Daru::DataFrame.new({x: t4_iterations, y: t4_losses}).plot(type: :line, x: :x, y: :y) do |plot, diagram|
  plot.x_label "Iteration"
  plot.y_label "Cumulative Loss"
end

#<Nyaplot::Plot:0x000056414ff191b8 @properties={:diagrams=>[#<Nyaplot::Diagram:0x000056414fcc08d0 @properties={:type=>:line, :options=>{:x=>:x, :y=>:y}, :data=>"0b75a13d-9154-46bd-9c4d-274cc554fe2f"}, @xrange=[1, 39], @yrange=[1.584592728710167, 11.745]>], :options=>{:x_label=>"Iteration", :y_label=>"Cumulative Loss", :zoom=>true, :width=>700, :xrange=>[1, 39], :yrange=>[1.584592728710167, 11.745]}}>

## Question 3.1 (5 Points)

Let's learn the parameter of a Bernoulli distribution using the method of maximum likelihood. Consider the following dataset in which we are tossing a biased coin with probability $\mu$ of returning a success (1). There is an analytical solution to find this parameter $\mu$ given a dataset of successes and trials. Compute this analytical solution. 

Here the ```label``` field is either 0 or 1. 

### Dataset format
This is the format for the coin dataset. The format below will be used throughout most of the assignments. A dataset contains some extra details like the classes and names of features. The ```data``` entry is an array of examples containing ```features``` and a label. Notice that in this particular dataset, there aren't any "features", just a ```bias``` value.

In [16]:
puts JSON.pretty_generate(coin_dataset(1))

{
  "classes": {
  },
  "features": [
    "x"
  ],
  "data": [
    {
      "features": {
        "bias": 1.0
      },
      "label": 1.0
    }
  ]
}


In [17]:
coin_data = coin_dataset(1000)

def q31_binomial_parameter(coin_data)
  # BEGIN YOUR CODE
    num = coin_data["data"].length
    i =  0
    sum = 0
    while i < num do
     sum +=coin_data["data"][i]["label"]
      i+=1.0
    end
    return sum/num
  #END YOUR CODE
end

:q31_binomial_parameter

In [18]:
### TESTS ###

t31_coin_data = coin_dataset(10000)
assert_in_delta 0.77, q31_binomial_parameter(t31_coin_data), 5e-2

## Question 3.2 (5 Points)

Now, let's use the maximum likelihood function and gradient descent to find the same parameter value. Define the objective function for a binomial distribution for multiple samples. Remember that the ```label``` is the target value and every example has only one feature, ```bias```. Learn the weight for the ```bias``` feature should converge to $w_{bias} = \mu$.

In this first step, calculate the log likelihood function for the binomial distribution of $n$ examples.

In [19]:
class BinomialModel
  def func examples, w
    # BEGIN YOUR CODE
    
    i = 0
    sum = 0.0

    while i<examples.length do
      
      sum += examples[i]["label"]*Math.log(w["bias"])  +  (1-examples[i]["label"])*Math.log(1-w["bias"])
      i+=1
    end
    return -sum

    #END YOUR CODE
  end
end

:func

In [20]:
### TEST ###
t32_model = BinomialModel.new
t32_t1 = {"features" => {"bias" => 1.0}, "label" => 0.0}
t32_t2 = {"features" => {"bias" => 1.0}, "label" => 1.0}
t32_dataset = {
  "data" => [t32_t1, t32_t2]
}

assert_in_delta 1.469677, t32_model.func([t32_t1], {"bias" => 0.77}), 1e-3
assert_in_delta 0.261365, t32_model.func([t32_t2], {"bias" => 0.77}), 1e-3
assert_in_delta 1.731041, t32_model.func(t32_dataset["data"], {"bias" => 0.77}), 1e-3


## Question 3.3 (5 Points)

Calculate the gradient of the binomial log likelihood function over $n$ examples.

In [21]:
class BinomialModel
  def grad examples, w
    # BEGIN YOUR CODE
    g = Hash.new
    i = 0
    sum = 0.0
    adjust w
    mu = w["bias"]
    while i< examples.length do
      x = examples[i]["label"]
      sum+=(x/mu) -(1.0-x)/(1.0-mu)

      i+=1
    end
    g.store("bias",-sum) 
    
    #END YOUR CODE
    return g
  end
  
  ## Adjusts the parameter to be within the allowable range
  def adjust w
    w["bias"] = [[0.001, w["bias"]].max, 0.999].min
  end
end

:adjust

In [22]:
### TEST ###
t33_model = BinomialModel.new
t33_t1 = {"features" => {"bias" => 1.0}, "label" => 0.0}
t33_t2 = {"features" => {"bias" => 1.0}, "label" => 1.0}

t33_dataset = {"data" => [t33_t1, t33_t2]}

assert_in_delta 4.347826, t32_model.grad([t33_t1], {"bias" => 0.77})["bias"], 1e-3
assert_in_delta -1.29870, t32_model.grad([t33_t2], {"bias" => 0.77})["bias"], 1e-3
assert_in_delta 3.049124, t32_model.grad(t33_dataset["data"], {"bias" => 0.77})["bias"], 1e-3

## Question 3.4 (5 Points)

Putting the objective function to work, use gradient descent to find the parameter for the binomial distribution. You get to set the learning rate. Return the learning rate you have obtained which works well. You may have to try a few until you get it right.

Note that, while capable of returning the same value, this method reads the data many more times than the analytical solution.

### Set the learning rate
Here, set this function to return one number. For example, if you decide that the learning rate needs to be 1.234, implement the following. Note: 1.234 might not be the best choice.

```ruby
def q34_learning_rate
  1.234
end
```

In [23]:
def q34_learning_rate
  # BEGIN YOUR CODE
  0.0000009
  #END YOUR CODE
end

:q34_learning_rate

In [24]:
### TEST ###
t34 = BinomialModel.new
t34_w_init = {"bias" => rand}
t34_data = coin_dataset(10000)

t34_learning_rate = q34_learning_rate()

t34_total_loss = 0.0
t34_iterations = []
t34_losses = []
t34_w = t34_w_init
gradient_descent t34_data, t34_w_init, t34, t34_learning_rate, 0.001, 100 do |w, iter, loss|
  puts [iter, w, t34_losses[-1]].join("\t") if iter % 10 == 1
  t34_total_loss += loss
  t34_iterations << iter
  assert_true(iter > 0, "Make sure to increment iter before calling yield")
  t34_losses << t34_total_loss / iter.to_f
  t34_w = w
end


assert_true(t34_losses[-1] < 8000)
assert_true(t34_losses[-1] > 0)
assert_true(t34_iterations[-1] > 30)
assert_in_delta 0.77, t34_w["bias"], 5e-2

### Plot the cumulative loss per iteration
Daru::DataFrame.new({x: t34_iterations, y: t34_losses}).plot(type: :line, x: :x, y: :y) do |plot, diagram|
  plot.x_label "Iteration"
  plot.y_label "Cumulative Loss"
end

1	{"bias"=>0.4583321657276431}	
11	{"bias"=>0.5562088315481644}	6889.483693674285
21	{"bias"=>0.6257456506585226}	6496.091312352947
31	{"bias"=>0.675782285084037}	6230.227059151687
41	{"bias"=>0.711068673747158}	6045.616080606115
51	{"bias"=>0.7351084101302675}	5914.802548525477
61	{"bias"=>0.7508784732892031}	5820.058694867535
71	{"bias"=>0.7608799892513304}	5749.676198105762
81	{"bias"=>0.7670582287438735}	5695.955256398518
91	{"bias"=>0.7708043637932329}	5653.859982940111
101	{"bias"=>0.7730480696542399}	5620.081396628424


#<Nyaplot::Plot:0x000056415146a2d8 @properties={:diagrams=>[#<Nyaplot::Diagram:0x0000564151445190 @properties={:type=>:line, :options=>{:x=>:x, :y=>:y}, :data=>"c43250d9-5f24-4209-a9de-f37991e1dc96"}, @xrange=[1, 101], @yrange=[5617.0688518248935, 7427.90054237226]>], :options=>{:x_label=>"Iteration", :y_label=>"Cumulative Loss", :zoom=>true, :width=>700, :xrange=>[1, 101], :yrange=>[5617.0688518248935, 7427.90054237226]}}>

## Question 4.1 (10 Points)

The maximum likelihood method above reads the data multiple times and can benefit from prior knowledge in the form of a prior distribution for the parameter, $\mu$. Using the Beta distribution as the conjugate prior, implement the likelihood function and its gradient. Now we are learning three parameters altogether: $w_{bias} = \mu$, $\alpha$, $\beta$.

First, let's compute the closed form estimator for $\mu$ with a fixed $\alpha$ and $\beta$.

In [25]:
def q41_closed_form_beta_binomial(coin_data, alpha, beta)
  # BEGIN YOUR CODE
    i=0
    sum =0.0
    while i < coin_data["data"].length do
      sum += coin_data["data"][i]["label"]
      i+=1
    end
    return (sum+alpha)/(alpha+beta+coin_data["data"].length)

  #END YOUR CODE
end

:q41_closed_form_beta_binomial

In [26]:
### TEST ###
t41_coin_data = coin_dataset(10000)
assert_in_delta 0.77, q41_closed_form_beta_binomial(t41_coin_data, 7743, 10000 - 7743), 1e-1

## Question 4.2 (10 points)

Implement the negative log likelihood function for the beta + binomial values. Checkout this [Wikipedia](https://en.wikipedia.org/wiki/Beta_distribution#Maximum_likelihood) definition of the likelihood function. Remember we are interested in the negative log likelihood.

A special function is needed ```GSL::Sf::lnbeta(a, b)```.

In [27]:
class BetaBinomialModel
  def func dataset, w
    # BEGIN YOUR CODE
      adjust w
      alpha = w["alpha"]
      beta = w["beta"]
      if w[0].is_a? Numeric
        bias = w[0]
        w["bias"]=bias
      elsif w["bias"].is_a? Numeric
        bias = w["bias"]
        w["bias"]=bias
      end
      adjust w
      y = GSL::Sf::lnbeta(alpha, beta)
      i = 0 
      sum =0
      num = dataset.length
      while i<num do
 
        sum += (alpha-1.0)* Math.log(bias) + (beta-1.0)* Math.log(1.0-bias)

        i+=1
      end
     t32_model =  BinomialModel.new
     sum = sum-y*num
      return -sum+t32_model.func(dataset, {"bias" => bias})
    
    
    #END YOUR CODE
  end
  def adjust w
    w["bias"] = [[0.001, w["bias"]].max, 0.999].min
    w["beta"] = [0.0001, w["beta"]].max
  end
end

:adjust

In [28]:
### TESTS ###

t42 = BetaBinomialModel.new
srand 777 #seed random number generator
t42_data = coin_dataset(1000)["data"]
t42_w = Hash.new {|h,k| h[k] = 0.1}
t42_w["alpha"] = 7.0
t42_w["beta"] = 3.0

assert_in_delta 10373.126026759332, t42.func(t42_data, t42_w), 1e-2

## Question 4.3 (10 points)

Implement the negative log likelihood gradient for all the parameters. Checkout this [Wikipedia](https://en.wikipedia.org/wiki/Beta_distribution#Maximum_likelihood) definition of the likelihood function. Remember we are interested in the negative log likelihood. The gradient for the ```bias``` requires all examples in the dataset. However, the gradient for ```alpha``` and ```beta``` does not require a pass over the dataset.

A special function is needed ```GSL::Sf::psi(a + b)```.

In [29]:
class BetaBinomialModel
  def grad dataset, w
    # BEGIN YOUR CODE
    adjust w
    alpha = w["alpha"]
    beta = w["beta"]
    if w[0].is_a? Numeric
        bias = w[0]
        w["bias"]=bias
    elsif w["bias"].is_a? Numeric
        bias = w["bias"]
        w["bias"]=bias
    end
    
    g = Hash.new
    g.store("alpha",-(Math.log(bias)-(-GSL::Sf::psi(alpha+beta)+GSL::Sf::psi(alpha))) )
    g.store("beta",-(Math.log(1-bias)-(-GSL::Sf::psi(alpha+beta)+GSL::Sf::psi(beta))))
    model  = BinomialModel.new
    g.store("bias",model.grad(dataset, {"bias" => bias})["bias"] )
    return g
    #END YOUR CODE
  end
end

:grad

In [30]:
### TESTS ###

t43 = BetaBinomialModel.new
srand 777 #seed random number generator
t43_data = coin_dataset(1000)["data"]
t43_w = Hash.new {|h,k| h[k] = 0.1}
t43_w["alpha"] = 7.0
t43_w["beta"] = 3.0

t43_grad = t43.grad(t43_data, t43_w)

assert_in_delta -7902.2222222221935, t43_grad["bias"], 1e2
assert_in_delta 1.9236168390257913, t43_grad["alpha"], 1e-1
assert_in_delta -1.2236077383104214, t43_grad["beta"], 1e-1


## Question 4.4 (20 points)

Run the gradient descent by selecting the initial weights and learning rate. Try a few values. 

In [31]:
def q44_weights_and_learning_rate

  w={"alpha"=>5,"beta"=>4,"bias"=>0.1}
  lr = 0.00001
  
  #END YOUR CODE
  return [w, lr]
end

:q44_weights_and_learning_rate

In [32]:
### TEST ###
t44 = BetaBinomialModel.new
t44_data = coin_dataset(10000)

t44_w_init, t44_learning_rate = q44_weights_and_learning_rate()

t44_total_loss = 0.0
t44_iterations = []
t44_losses = []
t44_w = t34_w_init
gradient_descent t44_data, t44_w_init, t44, t44_learning_rate, 0.001, 100 do |w, iter, loss|
  puts [iter, w, t44_losses[-1]].join("\t") if iter % 10 == 1
  t44_total_loss += loss
  t44_iterations << iter
  t44_losses << t44_total_loss / iter.to_f
  t44_w = w
end


assert_true(t44_losses[-1] < 8000)
assert_true(t44_iterations[-1] > 30)
assert_in_delta 0.77, t44_w["bias"], 5e-2

### Plot the cumulative loss per iteration
Daru::DataFrame.new({x: t44_iterations, y: t44_losses}).plot(type: :line, x: :x, y: :y) do |plot, diagram|
  plot.x_label "Iteration"
  plot.y_label "Cumulative Loss"
end

1	{"alpha"=>4.999983319387165, "beta"=>4.000007791632939, "bias"=>0.8530000000000271}	
11	{"alpha"=>5.000022878583458, "beta"=>3.999940564007174, "bias"=>0.7777056784023381}	5232.15650987812
21	{"alpha"=>5.000061188204944, "beta"=>3.999878645124152, "bias"=>0.7777000010067855}	4675.042731068756
31	{"alpha"=>5.000099496574959, "beta"=>3.9998167281625765, "bias"=>0.7777000000001785}	4489.134105648042
41	{"alpha"=>5.000137803819699, "beta"=>3.9997548126810654, "bias"=>0.7777000000000002}	4396.047261470562
51	{"alpha"=>5.000176109939219, "beta"=>3.999692898679531, "bias"=>0.7777000000000002}	4340.089138087342
61	{"alpha"=>5.0002144149335574, "beta"=>3.999630986157967, "bias"=>0.7777000000000002}	4302.695379592603
71	{"alpha"=>5.0002527188027495, "beta"=>3.9995690751163657, "bias"=>0.7777000000000002}	4275.909833453372
81	{"alpha"=>5.00029102154683, "beta"=>3.9995071655547174, "bias"=>0.7777000000000002}	4255.754423406623
91	{"alpha"=>5.000329323165835, "beta"=>3.9994452574730164, "bias"=>0

#<Nyaplot::Plot:0x00005641515185e0 @properties={:diagrams=>[#<Nyaplot::Diagram:0x00005641514cb6a0 @properties={:type=>:line, :options=>{:x=>:x, :y=>:y}, :data=>"4afdf6f1-eda5-42e7-9a10-19f205313c4b"}, @xrange=[1, 101], @yrange=[4226.248525446593, 13030.491935858397]>], :options=>{:x_label=>"Iteration", :y_label=>"Cumulative Loss", :zoom=>true, :width=>700, :xrange=>[1, 101], :yrange=>[4226.248525446593, 13030.491935858397]}}>